In [ ]:
import os
import re
import json
import pickle
from datetime import timedelta

import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_percentage_error
import matplotlib.pyplot as plt
import seaborn as sns

from comet_ml.api import API, APIExperiment
from comet_ml.query import Tag

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

fontsize = 9
lw = 0.75
matplotlib.rc('font', **{'family': 'Times', 'size': fontsize})
matplotlib.rc('axes', **{'linewidth': 0.75, 'labelsize': fontsize})
matplotlib.rc('xtick', **{'labelsize': fontsize})
matplotlib.rc('ytick', **{'labelsize': fontsize})
matplotlib.rc('xtick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.minor', **{'width': lw, 'size':1.5})

In [ ]:
CORRECT_M_VALUES = True

In [ ]:
api = API(api_key = os.environ['COMET_API_KEY'])
workspace = 'danielelinaro'
project_name = 'inertia'
query = Tag('Sardinia')# & Tag('low_high_prediction')
experiments = api.query(workspace, project_name, query, archived=False)
n_experiments = len(experiments)
print(f'Found {n_experiments} experiments.')

In [ ]:
val_loss = {'low_high': 1000, 'continuous': 1000}
experiment = {}
for i,expt in enumerate(experiments):
    metrics = expt.get_metrics_summary()
    tags = expt.get_tags()
    key = 'low_high' if 'low_high_prediction' in tags else 'continuous'
    metric_names = [m['name'] for m in metrics]
    if 'val_loss' in metric_names:
        idx = metric_names.index('val_loss')
        loss = float(metrics[idx]['valueMin'])
        if loss < val_loss[key]:
            val_loss[key] = loss
            experiment[key] = expt
    print('[{:2d}] {} {:g}{}'.format(i+1, expt.id, loss, ' *' if key == 'low_high' else ''))
for key in experiment:
    print("Best experiment for key '{}': {}[...]. Validation loss: {:.4f}.".\
          format(key, experiment[key].id[:8], val_loss[key]))
experiment_IDs = {k: v.id for k,v in experiment.items()}
metrics = {k: v.get_metrics() for k,v in experiment.items()}

In [ ]:
val_loss = {k: np.array([float(m['metricValue']) for m in v if m['metricName'] == 'val_loss'])
            for k,v in metrics.items()}
loss = {k: np.array([float(m['metricValue']) for m in v if m['metricName'] == 'loss'])
        for k,v in metrics.items()}
epochs = {k: np.arange(v.size) for k,v in loss.items()}

In [ ]:
base_dir = '..'
experiments_path = os.path.join(base_dir, 'experiments', 'neural_network')
y,y_pred,MAPE = {},{},{}
for k,ID in experiment_IDs.items():
    experiment_path = os.path.join(experiments_path, ID)
    checkpoint_path = os.path.join(experiment_path, 'checkpoints')
    test_results_path = os.path.join(experiments_path, ID, 'test_results.pkl')
    test_results = pickle.load(open(test_results_path, 'rb'))
    y[k] = test_results['y_test']
    y_pred[k] = test_results['y_prediction']
    MAPE[k] = test_results['mape_prediction'][0]

In [ ]:
if CORRECT_M_VALUES:
    old = np.load(os.path.join('..','HEM_old.npz'))
    new = np.load(os.path.join('..','HEM.npz'))
    M_map = {o: n for o,n in zip(old['M'],new['M']) if o > 0}
    y_new = {}
    y_pred_new = {}
    for k in y:
        y_new[k] = np.zeros_like(y[k])
        y_pred_new[k] = np.zeros_like(y_pred[k])
        for i,old_val in enumerate(y[k]):
            idx = np.argmin(np.abs(old_val - old['M']))
            y_new[k][i] = M_map[old['M'][idx]]
            y_pred_new[k][i] = y_pred[k][i] + (y_new[k][i] - old_val)

In [ ]:
fig,ax = plt.subplots(3, 1, figsize=(3.25,4), height_ratios=(1,1,2))

light_gray = .8 + np.zeros(3)
dark_gray = .2 + np.zeros(3)
for i,k in enumerate(['low_high','continuous']):
    ax[i].plot(epochs[k], loss[k], color=light_gray, lw=1, label='Batch loss')
    ax[i].plot(epochs[k], val_loss[k], color=dark_gray, lw=1, label='Validation loss')
for a in ax[:2]:
    a.set_yscale('log')
    a.set_ylabel('Loss')
    a.set_ylim([1e-3, 1e1])
ax[1].set_xlabel('Epoch')
ax[0].legend(loc='upper right', frameon=False, fontsize=8)

if CORRECT_M_VALUES:
    limits = [0.25,0.60]
    ticks = np.r_[0.25 : 0.61 : 0.05]
else:
    limits = [0.13,0.47]
    ticks = np.r_[0.15 : 0.46 : 0.1]
ax[2].plot(limits, limits, ':', color=[.6,.6,.6], lw=2)
cmap = {'low_high': 'tab:red', 'continuous': np.zeros(3)}
ms = {'low_high': 6, 'continuous': 3}
coeff = 3
if CORRECT_M_VALUES:
    Y,Y_pred = y_new,y_pred_new
else:
    Y,Y_pred = y,y_pred
for k in ('low_high','continuous'):
    flag = True
    for m in np.unique(Y[k]):
        idx, = np.where(Y[k] == m)
        mean,std = Y_pred[k][idx].mean(),Y_pred[k][idx].std()
        ax[2].vlines(m, mean-coeff*std, mean+coeff*std, color=cmap[k], lw=1.25)
        if flag:
            ax[2].plot(m, mean, 's', color=cmap[k], ms=ms[k], markerfacecolor='w',
                       markeredgewidth=1.25, label=f'MAPE = {MAPE[k]:.2f}%')
            flag = False
        else:
            ax[2].plot(m, mean, 's', color=cmap[k], ms=ms[k], markerfacecolor='w',
                       markeredgewidth=1.25)


ax[2].set_xticks(ticks)
ax[2].set_yticks(ticks)
ax[2].set_xlabel(r'Momentum [GW$\cdot$s$^2$]')
ax[2].set_ylabel(r'Predicted momentum [GW$\cdot$s$^2$]')
ax[2].legend(loc='upper left', frameon=False, fontsize=8)
sns.despine()
fig.tight_layout(pad=0)
plt.savefig('CNN_prediction.pdf')